<a href="https://colab.research.google.com/github/Buyan-Kirill/practice-VTM-sem-8/blob/main/Tensors_task_7.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

$\Large{TT-operations}$

In [24]:
import numpy as np
def TT_to_tensor(tt_cores):
    result = tt_cores[0]
    for core in tt_cores[1:]:
        result = np.einsum('...i,ijk->...jk', result, core)
    return np.squeeze(result)

In [25]:
def TT_SVD(Tensor, eps=1e-10):
    d = len(Tensor.shape)
    cores = []
    T = Tensor.copy()
    r_prev = 1
    eps_d = eps / d

    for k in range(d - 1):
        n_k = Tensor.shape[k]
        T = T.reshape((r_prev * n_k, -1))

        U, S, V = np.linalg.svd(T, full_matrices=False)

        rank = np.sum(np.cumsum(S[::-1]**2) > eps_d**2)

        G = U[:, :rank].reshape((r_prev, n_k, rank))
        cores.append(G)

        T = np.diag(S[:rank]) @ V[:rank, :]
        r_prev = rank

    cores.append(T.reshape((r_prev, Tensor.shape[-1], 1)))
    return cores

In [26]:
X = np.random.rand(4, 3, 2, 5)

tt_cores = TT_SVD(X, eps=1e-8)

X_reconstructed = TT_to_tensor(tt_cores)
error = np.linalg.norm(X - X_reconstructed) / np.linalg.norm(X)
print("Ошибка восстановления: ", error)

Ошибка восстановления:  3.3638087501750517e-15


In [27]:
X = np.random.rand(20, 10, 13, 7)

tt_cores = TT_SVD(X, eps=1e-8)

X_reconstructed = TT_to_tensor(tt_cores)
error = np.linalg.norm(X - X_reconstructed) / np.linalg.norm(X)
print("Ошибка восстановления: ", error)

Ошибка восстановления:  7.06117118565229e-15


In [28]:
def TT_orthogonalize(tt_cores, mode):
    if mode not in ['left', 'right']:
        raise ValueError("Incorrect mode")

    cores = [G.copy() for G in tt_cores]
    d = len(cores)

    if mode == 'left':
        for k in range(d - 1):
            r_prev, n_k, r_k = cores[k].shape

            Q, R = np.linalg.qr(cores[k].reshape(-1, r_k))

            cores[k] = Q.reshape(r_prev, n_k, Q.shape[-1])

            cores[k + 1] = np.einsum('ij,jkl->ikl', R, cores[k + 1])
    else:
        for k in range(d - 1, 0, -1):
            r_k, n_k, r_next = cores[k].shape

            Q, R = np.linalg.qr(cores[k].reshape(r_k, -1).T)
            Q, R = Q.T, R.T

            cores[k] = Q.reshape(Q.shape[0], n_k, r_next)

            cores[k - 1] = np.einsum('ijk,kl->ijl', cores[k - 1], R)

    return cores

In [29]:
def test_orthogonality(TT_cores, mode):
    flag = True
    if mode == "left":
        for k in range(len(TT_cores) - 1):
            Gk = TT_cores[k].reshape(-1, TT_cores[k].shape[-1])
            flag = flag and np.allclose(Gk.T @ Gk, np.eye(Gk.shape[1]))
    elif mode == "rifght":
        for k in range(len(TT_cores) - 1, 0, -1):
            Gk = TT_cores[k].reshape(TT_cores[k].shape[0], -1)
            flag = flag and np.allclose(Gk @ Gk.T, np.eye(Gk.shape[0]))
    return flag

In [30]:
np.random.seed(42)
G1 = np.random.rand(1, 10, 5)
G2 = np.random.rand(5, 3, 8)
G3 = np.random.rand(8, 5, 1)
G4 = np.random.rand(7, 11, 1)
TT_cores = [G1, G2, G3, G4]

print("Ортогональность слева", end="\n\n")

print("Before orthogonalization")
if test_orthogonality(TT_cores, "left"):
    print("Orthogonal")
else:
    print("Not orthogonal")

TT_cores = TT_orthogonalize(TT_cores, mode="left")

print("\nAfter orthogonalization")
if test_orthogonality(TT_cores, "left"):
    print("Orthogonal")
else:
    print("Not orthogonal")

Ортогональность слева

Before orthogonalization
Not orthogonal

After orthogonalization
Orthogonal


In [31]:
np.random.seed(42)
G1 = np.random.rand(1, 10, 5)
G2 = np.random.rand(5, 3, 8)
G3 = np.random.rand(8, 5, 1)
G4 = np.random.rand(7, 11, 1)
TT_cores = [G1, G2, G3, G4]

print("Ортогональность справа", end="\n\n")

print("Before orthogonalization")
if test_orthogonality(TT_cores, "right"):
    print("Orthogonal")
else:
    print("Not orthogonal")

TT_cores = TT_orthogonalize(TT_cores, mode="right")

print("\nAfter orthogonalization")
if test_orthogonality(TT_cores, "right"):
    print("Orthogonal")
else:
    print("Not orthogonal")

Ортогональность справа

Before orthogonalization
Orthogonal

After orthogonalization
Orthogonal


In [32]:
def TT_compression(G, eps=1e-15):
    G = TT_orthogonalize(G, mode="left")
    G[0] = G[0].reshape(G[0].shape[1], G[0].shape[2])
    G[-1] = G[-1].reshape(G[-1].shape[0], G[-1].shape[1])

    u, s, G[-1] = np.linalg.svd(G[-1], full_matrices=False)
    singular_count = np.argwhere(np.cumsum(s[::-1] ** 2) / s[0] > eps)[0][0]
    rank = s.size - singular_count
    G[-1] = G[-1][:rank, :]
    U = u[:, :rank] * s[np.newaxis, :rank]

    for i in range(len(G) - 2, 0, -1):
        G[i] = np.einsum("mrt,tk->mrk", G[i], U)
        n, right_rank = G[i].shape[1], G[i].shape[2]
        G[i] = np.reshape(G[i], (-1, n * rank))
        u, s, G[i] = np.linalg.svd(G[i])
        singular_count = np.argwhere(np.cumsum(s[::-1] ** 2) / s[0] > eps)[0][0]
        rank = s.size - singular_count
        G[i] = G[i][:rank, :]
        G[i] = np.reshape(G[i], (-1, n, right_rank))
        U = u[:, :rank] * s[np.newaxis, :rank]

    G[0] = G[0] @ U
    G[0] = G[0].reshape(1, G[0].shape[0], G[0].shape[1])
    G[-1] = G[-1].reshape(G[-1].shape[0], G[-1].shape[1], 1)
    return G

In [33]:
np.random.seed(42)
G1 = np.random.rand(1, 1, 4)
G2 = np.random.rand(4, 2, 4)
G3 = np.random.rand(4, 3, 1)
TT_cores = [G1, G2, G3]

print ("TT ranks before TT-compression: ", end = " ")
for i in range(len(TT_cores) - 1):
    print(TT_cores[i].shape[-1], end = ", ")
print()

T1 = TT_to_tensor(TT_cores)

TT_compressed = TT_compression(TT_cores)

print ("TT ranks after TT-compression: ", end = " ")
for i in range(len(TT_compressed) - 1):
    print(TT_compressed[i].shape[-1], end = ", ")
print()

T2 = TT_to_tensor(TT_compressed)

print("Error: ", np.linalg.norm(T1 - T2))

TT ranks before TT-compression:  4, 4, 
TT ranks after TT-compression:  1, 2, 
Error:  1.2560739669470201e-15


In [34]:
np.random.seed(42)
G1 = np.random.rand(1, 4, 20)
G2 = np.random.rand(20, 7, 80)
G3 = np.random.rand(80, 6, 18)
G4 = np.random.rand(18, 9, 1)
TT_cores = [G1, G2, G3, G4]

print ("TT ranks before TT-compression: ", end = " ")
for i in range(len(TT_cores) - 1):
    print(TT_cores[i].shape[-1], end = ", ")
print()

T1 = TT_to_tensor(TT_cores)

TT_compressed = TT_compression(TT_cores)

print ("TT ranks after TT-compression: ", end = " ")
for i in range(len(TT_compressed) - 1):
    print(TT_compressed[i].shape[-1], end = ", ")
print()

T2 = TT_to_tensor(TT_compressed)

print("Error: ", np.linalg.norm(T1 - T2))

TT ranks before TT-compression:  20, 80, 18, 
TT ranks after TT-compression:  4, 28, 9, 
Error:  1.0086937953180811e-10
